In [2]:
# Se importa vectorizer de sklearn

from sklearn.feature_extraction.text import CountVectorizer
import numpy as np 
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# abrimos el datafrem de juegos
df=pd.read_csv(r'C:\Users\p2_ge\Documents\proyectos\PI-MLOPS\Juegos_steam.csv')

In [4]:
df.head()

,Id_juego,Nombre,Desarrollador,Año_lanzamiento,Género
0,761140,Lost Summoner Kitty,Kotoshiro,2018,"['Action', 'Casual', 'Indie', 'Simulation', 'S..."
1,643980,Ironbound,Secret Level SRL,2018,"['Free to Play', 'Indie', 'RPG', 'Strategy']"
2,670290,Real Pool 3D - Poolians,Poolians.com,2017,"['Casual', 'Free to Play', 'Indie', 'Simulatio..."
3,767400,弹炸人2222,彼岸领域,2017,"['Action', 'Adventure', 'Casual']"
4,773570,Log Challenge,N/D,2018,"Action, Casual, Indie, Sports"


In [8]:
#pasamos la columna de Género a un formato de string, para que get dummies reconozca los generos únicos
def norm(y):
    p=''
    l=y.split(',')
    for a in l:
        a=a.strip("[]' ")
        p=p+a+', '
    p.strip(', ')
    return p


df.Género=df.Género.apply(lambda x: norm(x))

In [10]:
df.drop_duplicates(inplace=True)

In [11]:
# Obtenemos los dummies de Genero
X=df.Género.str.get_dummies(', ')

In [12]:
# ponemos los id de los juegos como indices de los dummies
X=X.set_index(pd.Index(df.Id_juego))

In [13]:
# se aplica la similitud del coseno a nuestros vectores
similitud = cosine_similarity(X)

In [14]:
similitudes=pd.DataFrame(similitud,index=df.Id_juego,columns=df.Id_juego)

In [15]:
dffuncion=df[['Id_juego','Nombre']]

In [16]:
dffuncion.head()

,Id_juego,Nombre
0,761140,Lost Summoner Kitty
1,643980,Ironbound
2,670290,Real Pool 3D - Poolians
3,767400,弹炸人2222
4,773570,Log Challenge


In [18]:
similitudes.head()

Id_juego,761140,643980,670290,767400,773570,772540,774276,774277,774278,768800,...,771810,767590,747320,769330,745400,773640,733530,610660,658870,681550
Id_juego,,,,,,,,,,,,,,,,,,,,,
761140,1.000000,0.447214,0.600000,0.516398,0.67082,0.516398,0.447214,0.447214,0.447214,0.670820,...,0.516398,0.632456,0.316228,0.632456,0.670820,0.894427,0.774597,0.516398,0.632456,0.600000
643980,0.447214,1.000000,0.447214,0.000000,0.25000,0.000000,0.500000,0.500000,0.500000,0.250000,...,0.288675,0.353553,0.707107,0.353553,0.250000,0.500000,0.577350,0.288675,0.353553,0.223607
670290,0.600000,0.447214,1.000000,0.258199,0.67082,0.258199,0.894427,0.894427,0.894427,0.670820,...,0.258199,0.632456,0.316228,0.632456,0.447214,0.670820,0.516398,0.516398,0.632456,0.400000
767400,0.516398,0.000000,0.258199,1.000000,0.57735,0.666667,0.000000,0.000000,0.000000,0.288675,...,0.666667,0.408248,0.000000,0.408248,0.866025,0.288675,0.333333,0.000000,0.408248,0.516398
773570,0.670820,0.250000,0.670820,0.577350,1.00000,0.288675,0.500000,0.500000,0.500000,0.500000,...,0.577350,0.707107,0.353553,0.707107,0.750000,0.500000,0.577350,0.288675,0.707107,0.447214


In [19]:
def nombres(id):
    nombres=similitudes.loc[id].nlargest(6)[1:6].index.to_list()
    l=df['Nombre'][df['Id_juego'].isin(nombres)].to_list()
    return l

In [20]:
dffuncion['juegos']=df.Id_juego.apply(lambda x:nombres(x))

C:\Users\p2_ge\AppData\Local\Temp\ipykernel_17112\325731463.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dffuncion['juegos']=df.Id_juego.apply(lambda x:nombres(x))


In [21]:
dffuncion

,Id_juego,Nombre,juegos
0,761140,Lost Summoner Kitty,"[Pixel Puzzles 2: Anime, World of Cinema - Dir..."
1,643980,Ironbound,"[Shadow Hunter, Immortal Empire, Immortal Empi..."
2,670290,Real Pool 3D - Poolians,"[Pixel Puzzles Ultimate - Puzzle Pack: Rio, Pi..."
3,767400,弹炸人2222,"[Atomic Adam: Episode 1, Biozone, Luxor: 5th P..."
4,773570,Log Challenge,"[Canyon Capers - Rio Fever, Lighthockey, 12 or..."
...,...,...,...
32130,773640,Colony On Mars,"[Fate of the World: Tipping Point, Fate of the..."
32131,733530,LOGistICAL: South Africa,"[Puzzler World 2, iBomber Defense Pacific, Bum..."
32132,610660,Russian Roads,"[City Car Driving, Try Hard Parking, Car Mecha..."
32133,658870,EXIT 2 - Directions,"[Zen of Sudoku, Gumboy - Crazy Adventures™, RI..."


In [22]:
dffuncion.to_csv('consulta_modelo1.csv', index=False)

In [3]:
dffuncion.head()

,Id_juego,Nombre,juegos
0,761140,Lost Summoner Kitty,"['Pixel Puzzles 2: Anime', 'World of Cinema - ..."
1,643980,Ironbound,"['Shadow Hunter', 'Immortal Empire', 'Immortal..."
2,670290,Real Pool 3D - Poolians,"['Pixel Puzzles Ultimate - Puzzle Pack: Rio', ..."
3,767400,弹炸人2222,"['Atomic Adam: Episode 1', 'Biozone', 'Luxor: ..."
4,773570,Log Challenge,"['Canyon Capers - Rio Fever', 'Lighthockey', '..."


In [14]:
def recomendacion_juego( id_de_producto ):
    return dffuncion.juegos[dffuncion.Id_juego==id_de_producto]


In [15]:
recomendacion_juego(773570)

4    ['Canyon Capers - Rio Fever', 'Lighthockey', '...
Name: juegos, dtype: object